In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("mps") if torch.has_mps else torch.device("cpu")
print(f"Using device: {device}")

data = pd.read_csv('results_1.csv')

X = data.drop('direction', axis=1).values
y = data['direction'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # CrossEntropyLoss는 long 타입이 필요함
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# MLP 모델 정의
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

input_size = X_train.shape[1]
hidden_size = 64
output_size = 4

model = MLP(input_size, hidden_size, output_size).to(device)  # 모델을 MPS로 이동

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#모델 학습 함수
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()  # 모델을 학습 모드로 설정
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 MPS로 이동
            optimizer.zero_grad()  # 기울기 초기화
            outputs = model(inputs)  # 예측 값
            loss = criterion(outputs, labels)  # 손실 계산 (labels에 차원 추가하지 않음)
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트
            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# 모델 학습
train_model(model, train_loader, criterion, optimizer, num_epochs=100)

# 테스트 함수
def test_model(model, test_loader):
    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad():
        total_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 MPS로 이동
            outputs = model(inputs)
            loss = criterion(outputs, labels)  # 손실 계산
            total_loss += loss.item()

            # 예측 정확도 계산
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Loss: {total_loss/len(test_loader):.4f}')
    print(f'Accuracy: {100 * correct / total:.2f}%')

# 모델 평가
test_model(model, test_loader)


/var/folders/71/2_0ln3796jd9pxnlbjvpf1r80000gn/T/ipykernel_3396/3122838209.py:9: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps") if torch.has_mps else torch.device("cpu")


Using device: mps
Epoch [1/100], Loss: 0.8778
Epoch [2/100], Loss: 0.5876
Epoch [3/100], Loss: 0.4513
Epoch [4/100], Loss: 0.3483
Epoch [5/100], Loss: 0.2675
Epoch [6/100], Loss: 0.2082
Epoch [7/100], Loss: 0.1625
Epoch [8/100], Loss: 0.1297
Epoch [9/100], Loss: 0.1033
Epoch [10/100], Loss: 0.0816
Epoch [11/100], Loss: 0.0651
Epoch [12/100], Loss: 0.0526
Epoch [13/100], Loss: 0.0435
Epoch [14/100], Loss: 0.0355
Epoch [15/100], Loss: 0.0296
Epoch [16/100], Loss: 0.0245
Epoch [17/100], Loss: 0.0210
Epoch [18/100], Loss: 0.0180
Epoch [19/100], Loss: 0.0157
Epoch [20/100], Loss: 0.0137
Epoch [21/100], Loss: 0.0121
Epoch [22/100], Loss: 0.0107
Epoch [23/100], Loss: 0.0096
Epoch [24/100], Loss: 0.0085
Epoch [25/100], Loss: 0.0076
Epoch [26/100], Loss: 0.0068
Epoch [27/100], Loss: 0.0062
Epoch [28/100], Loss: 0.0056
Epoch [29/100], Loss: 0.0051
Epoch [30/100], Loss: 0.0046
Epoch [31/100], Loss: 0.0042
Epoch [32/100], Loss: 0.0038
Epoch [33/100], Loss: 0.0035
Epoch [34/100], Loss: 0.0032
Epoch

In [17]:
def check_predictions(model, test_loader):
    model.eval()  # 평가 모드로 전환
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())  # 예측 값 저장
            all_labels.extend(labels.cpu().numpy())    # 실제 레이블 저장
    
    # 실제 레이블과 예측 값 출력 (첫 10개만 예시로 출력)
    for i in range(100):
        print(f"Actual: {all_labels[i]}, Predicted: {all_preds[i]}")

# 예측 값과 실제 레이블 비교
check_predictions(model, test_loader)


Actual: 3, Predicted: 3
Actual: 2, Predicted: 2
Actual: 4, Predicted: 2
Actual: 0, Predicted: 0
Actual: 2, Predicted: 2
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 3, Predicted: 3
Actual: 3, Predicted: 3
Actual: 4, Predicted: 2
Actual: 1, Predicted: 1
Actual: 2, Predicted: 2
Actual: 3, Predicted: 3
Actual: 2, Predicted: 2
Actual: 4, Predicted: 2
Actual: 2, Predicted: 2
Actual: 2, Predicted: 2
Actual: 3, Predicted: 3
Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 4, Predicted: 2
Actual: 2, Predicted: 2
Actual: 0, Predicted: 0
Actual: 4, Predicted: 2
Actual: 1, Predicted: 1
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 2, Predicted: 2
Actual: 4, Predicted: 2
Actual: 3, Predicted: 3
Actual: 4, Predicted: 2
Actual: 1, Predicted: 1
Actual: 0, Predicted: 0
Actual: 3, Predicted: 3
Actual: 2, Predicted: 2
Actual: 2, Predicted: 2
Actual: 3, Predicted: 3
Actual: 0, Predicted: 0
Actual: 2, Predicted: 2
Actual: 4, Predicted: 2
Actual: 0, Predicted: 0
Actual: 2, Predi